My plan is to build the initial df.

Write a for-loop on a counter of 9, to have a total of 1000 posts.

Pull the 'created_utc' on the current oldest post, last on the current df, and make the 'before' param : that value.

Make next 100 posts a new df.
Append them to the original df.

Ultimately, I want to put this into a function so I can switch to different subreddits and pull submissions and comments.

In [1]:
import pandas as pd
import requests

In [2]:
url = 'https://api.pushshift.io'

In [6]:
subm_end = '/reddit/search/submission'
comment_end = '/reddit/search/comment'

Building the initial df.

In [5]:
params = {
    'subreddit': 'startrek',
    'size': 100
}

In [7]:
res = requests.get(url+subm_end, params)

In [8]:
res.status_code

200

In [9]:
data = res.json()

In [10]:
posts = data['data']

In [11]:
len(posts)

100

In [12]:
df = pd.DataFrame(posts)

In [13]:
df.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_is_blocked,author_patreon_flair,...,post_hint,preview,thumbnail_height,thumbnail_width,url_overridden_by_dest,media,media_embed,secure_media,secure_media_embed,author_cakeday
0,[],False,Samsons_girl,None,[],None,text,t2_2wttoofq,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,Falkoro,None,[],None,text,t2_i786y,False,False,...,link,"{'enabled': False, 'images': [{'id': 'LrWDPGyA...",125.0,140.0,https://www.reddit.com/r/vegan/comments/vi5lvt...,NaN,NaN,NaN,NaN,NaN
2,[],False,Equivalent_Pen_5613,None,[],None,text,t2_bkp79451,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,malloryduncan,None,[],None,text,t2_2e40duwf,False,False,...,link,"{'enabled': False, 'images': [{'id': 'rpzDE308...",70.0,140.0,https://people.com/tv/bill-nye-liza-mundy-wedd...,NaN,NaN,NaN,NaN,NaN
4,[],False,Aezetyr,None,[],None,text,t2_82rj4jao,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df.shape

(100, 70)

In [17]:
# get last time 
df.iloc[-1]['created_utc']

1655755319

I referenced this [stackoverflow](https://stackoverflow.com/questions/28423964/for-loop-should-loop-9-times-instead-loops-once) to see using a counter to iterate through the for-loop 9 times (to get 1000 submissions in the df).

I needed [this website](https://techoverflow.net/2020/05/24/how-to-get-last-row-of-pandas-dataframe/) to remind me to use `.iloc[]` to get the last row (my previous stab at it having failed).

In [27]:
# I could start with an outer for loop that tests whether df is empty or not. if it is, it establishes the initial df
# else it runs the below

# Write a for-loop on a counter of 9, to have a total of 1000 posts.

for i in range(9): # I want to write my function so you can change the number of iterations in the function.
    
    params = {
        'subreddit': 'startrek', #ultimately, I want to make this a variable in the function.
        'size': 100,
        'before': df.iloc[-1]['created_utc']
    }
    res = requests.get(url+subm_end, params) #I want to make the +subm_end part a variable in the function,
                                                # default to subm_end, I think.
    #add a counter here, instead of printing the res.status_code, count the cycles and add to status code.
    if res.status_code == 200:
        print(res.status_code)
        data = res.json()
        posts = data['data']
        newdf = pd.DataFrame(posts)
        df = pd.concat([df, newdf], ignore_index = True) 

# Make next 100 posts a new df. Append them to the original df.
    else:
        print('ERROR: STATUS CODE NOT 200')

200
200
200
200
200
200
200
200
200


In [28]:
df.shape

(999, 73)

Why more columns? Why 999, not 1000? Maybe the first one gets overwritten somehow?

In [29]:
df.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_is_blocked,author_patreon_flair,...,thumbnail_width,url_overridden_by_dest,media,media_embed,secure_media,secure_media_embed,author_cakeday,distinguished,author_flair_background_color,author_flair_text_color
0,[],False,Samsons_girl,None,[],None,text,t2_2wttoofq,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,Falkoro,None,[],None,text,t2_i786y,False,False,...,140.0,https://www.reddit.com/r/vegan/comments/vi5lvt...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,Equivalent_Pen_5613,None,[],None,text,t2_bkp79451,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,malloryduncan,None,[],None,text,t2_2e40duwf,False,False,...,140.0,https://people.com/tv/bill-nye-liza-mundy-wedd...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,Aezetyr,None,[],None,text,t2_82rj4jao,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df = []

I got this error `SyntaxError: non-default argument follows default argument` on my initial attempt. I didn't realize the default arguments had to all come at the end. [This stackoverflow](https://stackoverflow.com/a/39942121) answer clarified that for me.

In [51]:
def build_reddit_df(dfname, subreddit, desired_size, endpoint = '/reddit/search/submission', limit = 100):
    '''
    This function is designed to use pushshift API to build a dataframe of specified size 
    filled with data from the specified subreddit. It starts with the most recent post and works backwards.

    df = name of dataframe you want to build

    endpoint = your desired endpoint. Defaults to '/reddit/search/submission' for submissions (main post) and
       '/reddit/search/comment' for comments at the time of the writing of this function (6/22/2022) 

    subreddit = the subreddit you'd like to scrape

    limit = the limit for number of posts that can be pulled at once. The default is 100, the maximum
        allowed at the time of the writing of this function (6/22/2022)

    desired_size = the total number of posts you'd like to have in the dataframe
    '''
    url = 'https://api.pushshift.io'+endpoint
    
    counter = 0
    
    dfname = [] #establish with certainty that the new dataframe name is empty.
    
    for i in range(2):
   
        if len(dfname) == 0:
            params = {
                'subreddit': subreddit,
                'size': limit
            }
            res = requests.get(url, params)
            if res.status_code == 200:
                data = res.json()
                posts = data['data']
                dfname = pd.DataFrame(posts)
                counter += 1
            else:
                print(f'ERROR: status code not 200. Failure occured on loop number {counter+1}')

        else: # after the df has been established.
            for i in range(int(round(desired_size)/(limit))-1):
                params = {
                    'subreddit': subreddit,
                    'size': limit,
                    'before': dfname.iloc[-1]['created_utc']
                }
                res = requests.get(url, params)

                if res.status_code == 200:
                    data = res.json()
                    posts = data['data']
                    newdf = pd.DataFrame(posts)
                    dfname = pd.concat([dfname, newdf], ignore_index = True)
                    counter +=1

                else:
                    print('ERROR: status code not 200. Failure occured on loop number {counter+1}')
    
    return dfname

In [52]:
build_reddit_df('startrek', 'startrek', 1000)

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_is_blocked,author_patreon_flair,...,preview,secure_media,secure_media_embed,thumbnail_height,thumbnail_width,url_overridden_by_dest,author_cakeday,distinguished,author_flair_background_color,author_flair_text_color
0,[],False,Pitbull_Petter-9001,None,[],None,text,t2_63i2wphj,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,dccs120,None,[],None,text,t2_5vvlrrr,False,False,...,"{'enabled': False, 'images': [{'id': 'Ei-mThqC...","{'oembed': {'author_name': 'AlphaYellow', 'aut...","{'content': '&lt;iframe width=""356"" height=""20...",105.0,140.0,https://www.youtube.com/watch?v=kp4YXez_6k0,NaN,NaN,NaN,NaN
2,[],False,Samsons_girl,None,[],None,text,t2_2wttoofq,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,Falkoro,None,[],None,text,t2_i786y,False,False,...,"{'enabled': False, 'images': [{'id': 'LrWDPGyA...",NaN,NaN,125.0,140.0,https://www.reddit.com/r/vegan/comments/vi5lvt...,NaN,NaN,NaN,NaN
4,[],False,Equivalent_Pen_5613,None,[],None,text,t2_bkp79451,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,[],False,Chuky240514,None,[],None,text,t2_8kzanmp8,False,False,...,"{'enabled': False, 'images': [{'id': 'SYJi_e_M...",NaN,NaN,94.0,140.0,https://www.salon.com/2022/05/23/voyager-1-data/,NaN,NaN,NaN,NaN
995,[],False,Equivalent_Squash604,None,[],None,text,t2_90crjogt,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,[],False,SwordfishCalm9013,None,[],None,text,t2_i52e6qmq,False,False,...,"{'enabled': False, 'images': [{'id': 'qvH1ZXdd...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,[],False,ShiroHachiRoku,None,[],None,text,t2_be4yl,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
